In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.applications.inception_v3 import InceptionV3
#from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

#from keras import backend as K
#K.set_image_dim_ordering('th')

import numpy as np
import pandas as pd
#import h5py

import matplotlib.pyplot as plt

In [6]:
inc_model = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_shape=((150, 150, 1)))

In [ ]:
datagen = ImageDataGenerator(rescale=1./255) # масштабируем именно значения каждого пикселя

In [ ]:
#Пропускаем через свёртучнуя чать сети изображения (как "train", так и "test"), чтобы выделить массивы наиболее важных признаков

train_generator = datagen.flow_from_directory('data/img_train/', #Важно разложить по папкам объекты классов
                                        target_size=(150, 150), #Размер изображения (высота и ширина в пикселях)
                                        batch_size=32,
                                        class_mode=None,
                                        shuffle=False) #не перемешиваем, т.к. эти данные будем прогонять через уже обученную
                                                       #сеть. Далее будет удобно работать с индексами классов при загрузке
                                                       #массивов numpy для обучения полносвязанных слоёв
                                                       # keep data in same order as labels

        
#Нужно настроить: color_mode=не "rgb" (3 канала), а "grayscale" (1 канал)
#Трансофрмацию валидационных данных не нужно делать
test_generator = datagen.flow_from_directory('data/img_test/',
                                                               target_size=(150, 150), #разобраться с количеством тестовых
                                                                                       #нужно, чтобы их было 25%, ане 50%
                                                               batch_size=50,
                                                               class_mode=None,
                                                               shuffle=False)

In [ ]:
features_train = inc_model.predict_generator(train_generator, 2000,  verbose=1) #2000 значит, что мы предскажем 
                                                                                #только первые 2000 элементов из папки
np.save(open('bottleneck_features/bn_features_train.npy', 'wb'), features_train) #'wb' - открыть/создать
                                                                                 #файл для двоичной записи

features_test = inc_model.predict_generator(test_generator, 2000,  verbose=1) #Настроить количество предсказаний
np.save(open('bottleneck_features/bn_features_test.npy', 'wb'), features_test)

In [ ]:
train_data = np.load(open('bottleneck_features/bn_features_train.npy', 'rb'))
train_labels = np.array([0] * 1000 + [1] * 1000) 

test_data = np.load(open('bottleneck_features/bn_features_test.npy', 'rb'))
test_labels = np.array([0] * 1000 + [1] * 1000)

In [ ]:
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:])) #слой ввода (input)
fc_model.add(Dense(64, activation='relu', name='dense_one')) #можно попробовать функцию активации 'selu'
fc_model.add(Dropout(0.5, name='dropout_one')) #выбрасываем сигналы от половины нейронов
fc_model.add(Dense(64, activation='relu', name='dense_two'))
fc_model.add(Dropout(0.5, name='dropout_two'))
fc_model.add(Dense(1, activation='sigmoid', name='output')) #можно использовать 'softmax' для многоклассовой классификации
# количество нейронов в выходном слое равно количеству классов (ззаписываем максимальный номер класса)

fc_model.compile(optimizer='rmsprop', #поиск минимума (может быть 'adam', 'sgd', 'rmsprop')
              loss='binary_crossentropy', # 'categorical_crossentropy'
              metrics=['accuracy'])